# Tugas Crawling

In [21]:
!pip install BeautifulSoup4

In [22]:
from typing_extensions import final
import requests
from bs4 import BeautifulSoup
import csv

# mengambil data dari website
def get_data (url):
  page = requests.get(url)
  if page.status_code == 200:
    page_html = page.content
    page_soup = BeautifulSoup(page_html, 'html.parser')
    data = []

    container_header = page_soup.find('div', {'style' : 'float:left; width:540px;'})
    judul = container_header.find('a', class_='title').text.strip().replace('\r\n', '')
    person = container_header.find_all('span')
    abstrak = page_soup.find('p', {'align' : 'justify'}).text.strip().replace('\r\n', '')
    data.append(judul)
    for i in person:
      split_text = i.text.strip().split(':')
      data.append(split_text[1])
    data.append(abstrak)
    return (data)

In [23]:
# mengambil link selengkapnya dan juga menyimpan data
def crawl_pta_trunojoyo(url):
  response = requests.get(url)

  if response.status_code == 200:
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    next_button = soup.find_all('a', class_='pag_button')
    for i in next_button:
      if i.text.strip() == '>' :
        next_page_link = i.get('href')
    data = []


    content_page = soup.find('ul', class_='items')
    link_content = content_page.find_all('li')
    for detail in link_content:
      link_view = detail.find('a', class_='gray button').get('href')
      data.append(get_data(link_view))
    return(data, next_page_link)

In [24]:
# Membuat sebuah fungsi main untuk memanggil berapa page untuk crawling data
def main(url, num_page):
  next_page_link = url
  final_crawl = []
  for i in range(num_page):
    hasil_crawl, next_page_link = crawl_pta_trunojoyo(next_page_link)
    final_crawl+=hasil_crawl

  # Membuka file CSV untuk penulisan
  with open('pta_trunojoyo.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Judul', 'Penulis', 'Dosen Pembimbing I', 'Dosen Pembimbing II', 'Abstrak'])
    csvwriter.writerows(final_crawl)
  print(f'Data Berhasil Di Simpan Dengan Jumlah {len(final_crawl)} data')


In [25]:
main('https://pta.trunojoyo.ac.id/c_search/byprod/10', 1)

Data Berhasil Di Simpan Dengan Jumlah 5 data
